# Working with Sample Data

### Import the packages and data 

In [6]:
from IPython.display import Image
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import warnings
warnings.simplefilter("ignore")
%matplotlib inline

matplotlib.rcParams['figure.figsize'] = [12, 12]
pd.set_option("precision", 5)

import pyarrow.parquet as pq


In [8]:
data = pd.read_csv('Data/customer_data.csv')
#data = table.to_pandas()

In [12]:
del data["Unnamed: 0"]

In [13]:
data.head()

,UniqueID,FrequencyofVisits,Age,Sex,Payer,Facility
1,547c0d52bbfe1b93b9411d7b55f3bd82,3,37.0,Feminino,ADVANCECARE,HCP
2,1770ad573874abe83cd04bfaf89c8eb6,2,37.0,Feminino,ENTIDADES COM DESCONTO,CCB
3,11b9573f2fc9dd49594cd1f301cb7be6,16,37.0,Feminino,MÉDIS,HCS
4,84fdd6d40ab2a9b95f3c957237225d33,1,37.0,Feminino,PARTICULARES,CCB
5,0ea39c0f610f6363aace9e4c5ba4c5d2,5,37.0,Feminino,MÉDIS,HCIS


In [ ]:
##data = data[0:1000]

## Working with K-means

### Loading the data 

In [14]:
data.dtypes

UniqueID              object
FrequencyofVisits      int64
Age                  float64
Sex                   object
Payer                 object
Facility              object
dtype: object

In [15]:
data = data.drop(columns="UniqueID")

In [24]:
data.isnull().values.any()

False

In [17]:
numerical_data = data.select_dtypes(np.number)
categorical_data = data.select_dtypes([object, "category"])

In [18]:
from sklearn.preprocessing import MinMaxScaler

numerical_data_normalized = MinMaxScaler().fit_transform(numerical_data)
numerical_data_normalized = pd.DataFrame(
                    numerical_data_normalized,
                    columns=numerical_data.columns) 

In [19]:
categorical_data_codified = pd.get_dummies(
                                    categorical_data, 
                                    drop_first=True
)

In [20]:
data_processed = pd.concat([
                            numerical_data_normalized,
                            categorical_data_codified
                        ], axis=1
)
data_processed.head(5)

,FrequencyofVisits,Age,Sex_Masculino,Sex_Missing,Payer_ADMG,Payer_ADSE,Payer_ADVANCECARE,Payer_ALLIANZ,Payer_ARS,Payer_ASSISTÊNCIA INTERNACIONAL,...,Facility_['HCIS' 'HCS'],Facility_['HCIS' 'HCV'],Facility_['HCIS' 'ICDT'],Facility_['HCP' 'HCS'],Facility_['HCP' 'HCV' 'ICDT'],Facility_['HCP' 'HCV'],Facility_['HCP' 'ICDT'],Facility_['HCS' 'HCV'],Facility_['HCS' 'ICDT'],Facility_['HCV' 'ICDT']
0,0.00313,0.34906,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0.00156,0.34906,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.02347,0.34906,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.00000,0.34906,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.00626,0.34906,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [25]:
data_processed.isnull().values.any()

True

In [21]:
data_processed.shape

(810341, 589)

### K-means

In [22]:
from sklearn.cluster import KMeans

In [27]:
data_processed= data_processed.dropna()

In [ ]:
estimator_kmeans = KMeans(random_state=42, n_clusters=3)
estimator_kmeans.fit(data_processed)

In [ ]:
cluster_labels = estimator_kmeans.labels_
#cluster_labels

In [ ]:
#seperate the data into test and train and use train all the way and use ".predict" for the test

In [ ]:
centroids = estimator_kmeans.cluster_centers_

In [ ]:
centroids.shape

In [ ]:
estimator_kmeans.inertia_

In [ ]:
from sklearn.metrics import euclidean_distances

In [ ]:
centroid_distances = euclidean_distances(centroids)

In [ ]:
centroid_distances.shape

In [ ]:
estimator_kmeans.labels_.shape

In [ ]:
data["Cluster"] = cluster_labels

In [ ]:
data.head()

In [ ]:
def cluster_summary(cluster_id):
    cluster = data[Cluster==cluster_id]
    cluster_summary = cluster[categorical_data.columns].mode().to_dict(orient="records")[0]
    cluster_summary.update(cluster.mean().to_dict())
    cluster_summary["cluster_id"] = cluster_id
    return cluster_summary

In [ ]:
cluster = data[data.Cluster==2]
cluster[categorical_data.columns].mode().to_dict(orient="records")

In [ ]:
cluster_summary(1)

In [ ]:
# Distance between centroid of cluster 0 and 2 
centroid_distances[0,2]

In [ ]:
def cluster_comparison(*cluster_ids):
    summaries = []
    for cluster_id in cluster_ids:
        summaries.append(cluster_summary(cluster_id))
    return pd.DataFrame(summaries).set_index("cluster_id").T

In [ ]:
cluster_comparison(0,1,2)

In [ ]:
def kmeans_cluster(df, n_clusters=3):
    model = KMeans(n_clusters=n_clusters, random_state=42)
    clusters = model.fit_predict(df)
    cluster_results = df.copy()
    cluster_results['Cluster'] = clusters
    return cluster_results

def graph_summary_clustering(results):
    cluster_size = results.groupby(['Cluster']).size().reset_index()
    cluster_size.columns = ['Cluster', 'Count']
    cluster_means = results.groupby(['Cluster'], as_index=False).mean()
    cluster_summary = pd.merge(cluster_size, cluster_means, on='Cluster')
    cluster_summary = cluster_summary.drop(["Count"], axis=1).set_index("Cluster")
    return cluster_summary[sorted(cluster_summary.columns)]

In [ ]:
cluster_results = kmeans_cluster(data_processed, 3)
cluster_summary = graph_summary_clustering(cluster_results);
cluster_summary

In [ ]:
matplotlib.rcParams['figure.figsize'] = [14, 14]
import seaborn as sns
sns.heatmap(cluster_summary.transpose(), annot=True);

### Ideal number of clusters?

In [ ]:
from scipy.spatial.distance import cdist

In [ ]:
total_variance = cdist(
    XA=data_processed, 
    XB=np.array([data_processed.mean()]))

In [ ]:
total_variance_sum = total_variance.sum()

In [ ]:
#functions to calculate the Within-Cluster sum of squares (WSS)
def cluster_variance(cluster_id, cluster_centroid, cluster_labels):
    cluster_elements = data_processed[cluster_labels==cluster_id]
    return cdist(XA=cluster_elements, XB=np.array([cluster_centroid])).sum()

#explained variance (defined as the reduction of variance compared to the total variance,...
# ... which would be the variance where k=1).
def variance_measure(estimator_kmeans, total_variance):
    clusters_labels = estimator_kmeans.labels_
    wss = 0
    for i, cluster_id in enumerate(np.unique(clusters_labels)):
        cluster_centroid = estimator_kmeans.cluster_centers_[i]
        wss += cluster_variance(cluster_id, cluster_centroid, clusters_labels)
    return (total_variance-wss) / total_variance

In [ ]:
# Function that returns the inertia.
def inertia_measure(estimator_kmeans):
    return estimator_kmeans.inertia_

In [ ]:
# Create a function that evaluates the intertia and percentage variance explained for a value of K
def evaluate_k_kmeans(k, scoring, **kwargs):
    scoring_measure = None
    if scoring=="inertia":
        scoring_measure = inertia_measure
    elif scoring=="variance":
        scoring_measure = variance_measure
        
    estimator_kmeans = KMeans(random_state=42, n_clusters=k)
    estimator_kmeans.fit(data_processed)
    return scoring_measure(estimator_kmeans, **kwargs)

In [ ]:
results_k = {}
range_k = [1,2,3,4,5,6,7,8,9] #range_k gives the number of clusters to evaluate their inertia and variance
for k in range_k:
    results_k[k] = (
        evaluate_k_kmeans(k, "inertia"), 
        evaluate_k_kmeans(k, "variance", total_variance=total_variance_sum), 
    )

In [ ]:
fig, ax1 = plt.subplots()

ax1.plot(
    [c[0] for c in results_k.items()],
    [c[1][0] for c in results_k.items()], label="inertia", color="red")
ax1.set_ylabel('inertia', color="red")


ax2 = ax1.twinx()
ax2.plot(
    [c[0] for c in results_k.items()],
    [c[1][1] for c in results_k.items()], label="percentage variance explained", color="blue")
ax2.set_ylabel('percentage variance', color='blue')

plt.xlabel("K")
plt.legend()
plt.title("Percentage variance / Inertia by. K");

In [ ]:
sse = {}
for k in range(1, 10):
   kmeans = KMeans(n_clusters=k).fit(data_processed)
   #print(data["clusters"])
   sse[k] = kmeans.inertia_ # Inertia: Sum of distances of samples to their closest cluster center
plt.figure()
plt.plot(list(sse.keys()), list(sse.values()))
plt.xlabel("Number of cluster")
plt.ylabel("SSE")
plt.show()

## Change the number of clusters 

In [ ]:
clusterer = KMeans(n_clusters=6)
clusterer.fit(data_processed)

In [ ]:
from sklearn.model_selection import cross_val_score
from sklearn.metrics import silhouette_score

In [ ]:
def silhouette_score_cv(clusterer, X, y=None):
    cluster_labels = clusterer.fit_predict(X)
    return silhouette_score(X, cluster_labels)

In [ ]:
cross_val_score(clusterer, data_processed, y=None, 
                scoring=silhouette_score_cv, cv=3).mean() #a small value means it is well matched

In [ ]:
pd.Series(clusterer.labels_).value_counts()

In [ ]:
new_clusters = clusterer.labels_
data["Cluster"] = new_clusters
data.head()